### Spark session configuration
This cell sets Spark session settings to enable _Verti-Parquet_ and _Optimize on Write_. More details about _Verti-Parquet_ and _Optimize on Write_ in tutorial document.

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 3, Finished, Available, Finished)

#### Read a Delta Table into a Spark Dataframe

In [3]:
df_factinternetssales = spark.read.table("FactInternetSales2014") 
display(df_factinternetssales.head(5))

StatementMeta(, c88099a9-f92d-4e30-9e1f-d9796a74e97e, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5feb4bd4-f6e8-431c-a965-63a309597f9a)

## Create a view in the Data Lakehouse

In [3]:
%%sql   

CREATE OR REPLACE VIEW v_sales_info as
SELECT s.*, d.*, t.SalesTerritoryRegion, t.SalesTerritoryCountry 
FROM FactInternetSales2014   s
JOIN DimDate             d 
ON s.OrderDateKey = d.DateKey
JOIN DimSalesTerritory   t 
ON s.SalesTerritoryKey = t.SalesTerritoryKey


StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql

SELECT * FROM v_sales_info LIMIT 5

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 6, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 47 fields>

## Load the view we created into a Spark Dataframe

In [5]:
df_salesinfo = spark.read.table("v_sales_info")
display(df_salesinfo.head(3))

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 99115a71-72e4-43bd-b3d2-fb9d18b32179)

## Get some stats on the data...

In [6]:
display(df_salesinfo.select('SalesAmount', 'TotalProductCost').describe())

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 945a3777-c36a-474f-a9d8-a532bcbe5deb)

In [7]:
from pyspark.sql.functions import col, round

df_sales_summary = (df_salesinfo.select('FiscalYear', 'SalesTerritoryRegion', 'SalesTerritoryCountry', 'SalesAmount')
.groupBy('FiscalYear','SalesTerritoryRegion', 'SalesTerritoryCountry')
.sum('SalesAmount')
.withColumnRenamed("sum(SalesAmount)", "SumOfSalesAmount") 
.orderBy('FiscalYear','SalesTerritoryRegion', 'SalesTerritoryCountry'))

display(df_sales_summary)

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 22d5d656-ad12-4bc9-86dc-c1a5724d7d54)

In [8]:
df_sales_summary.createOrReplaceTempView("v_sales_summary") 

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 10, Finished, Available, Finished)

In [9]:
%%sql

SELECT * FROM v_sales_summary

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 11, Finished, Available, Finished)

<Spark SQL result set with 8 rows and 4 fields>

In this cell, you are reading from the temporary Spark view created in the previous cell and and finally writing it as delta table in the _Tables_ section of the lakehouse.

### Save the data to a Delta Table for future use.

In [10]:
%%sql

CREATE OR REPLACE TABLE aw_sales_summary AS
SELECT * FROM v_sales_summary

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

### Confirm the table is there.

In [11]:
%%sql

SELECT * FROM aw_sales_summary

StatementMeta(, 9117cb4c-1002-4687-8226-ec6ae2dad85c, 13, Finished, Available, Finished)

<Spark SQL result set with 8 rows and 4 fields>